In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import csv
import pprint
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 

In [2]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\ATL08"   # I should change this so I can work from github?
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
# select first MODIS file
ATL08Files = glob.glob('*ATL08*')
ATL08 = h5py.File(ATL08Files[0], 'r')

In [4]:
mean_canopy = ATL08['/gt1r/land_segments/canopy/h_mean_canopy'][:]
max_canopy = ATL08['/gt1r/land_segments/canopy/h_max_canopy'][:]
min_canopy = ATL08['/gt1r/land_segments/canopy/h_min_canopy'][:]
median_canopy = ATL08['/gt1r/land_segments/canopy/h_median_canopy'][:]
canopy_openness = ATL08['/gt1r/land_segments/canopy/canopy_openness'][:]
latitude = ATL08['/gt1r/land_segments/latitude'] [:]
longitude = ATL08['/gt1r/land_segments/longitude'][:]

df08=pd.DataFrame({'latitude':latitude,'longitude':longitude, 'mean_canopy': mean_canopy, 
                   'max_canopy': max_canopy, 'min_canopy': min_canopy, 'median_canopy': median_canopy,
                  'canopy_openness': canopy_openness})

In [5]:
np.savetxt("D:\\masters_data\\output\\csv\\df08_points.csv", df08, delimiter=",")

In [6]:
d = {}
number = 0


for file in ATL08Files:
    FILE = h5py.File(ATL08Files[number], 'r')
    max_canopy = FILE['/gt1r/land_segments/canopy/h_max_canopy'][:]
    min_canopy = FILE['/gt1r/land_segments/canopy/h_min_canopy'][:]
    median_canopy = FILE['/gt1r/land_segments/canopy/h_median_canopy'][:]
    canopy_openness = FILE['/gt1r/land_segments/canopy/canopy_openness'][:]
    mean_canopy = FILE['/gt1r/land_segments/canopy/h_mean_canopy'][:]
    
    latitude = FILE['/gt1r/land_segments/latitude'] [:]
    longitude = FILE['/gt1r/land_segments/longitude'][:]
    
    
    d[number] = pd.DataFrame({'latitude':latitude,'longitude':longitude, 'mean_canopy': mean_canopy,
                             'max_canopy': max_canopy, 'min_canopy': min_canopy, 'median_canopy': median_canopy,
                             'canopy_openness': canopy_openness})
    number += 1

In [7]:
all_points = pd.concat(d)

In [8]:
all_points = all_points[all_points.longitude > 14]
all_points = all_points[all_points.longitude < 27.6]
all_points = all_points[all_points.latitude > 2]
all_points = all_points[all_points.latitude < 11]

In [9]:
print(all_points)

            latitude  longitude   mean_canopy    max_canopy    min_canopy  \
0   16347  10.808227  18.664986  4.863425e+00  1.008167e+01  5.119019e-01   
    16348  10.807324  18.664894  5.473944e+00  1.064835e+01  6.545715e-01   
    16349  10.806422  18.664801  4.802375e+00  1.090176e+01  5.988159e-01   
    16350  10.523349  18.636093  4.987176e+00  9.200226e+00  5.248718e-01   
    16351  10.515229  18.635271  3.402823e+38  3.402823e+38  3.402823e+38   
...              ...        ...           ...           ...           ...   
540 10092  10.995555  27.352436  3.402823e+38  3.402823e+38  3.402823e+38   
    10093  10.996457  27.352345  3.402823e+38  3.402823e+38  3.402823e+38   
    10094  10.997359  27.352249  3.402823e+38  3.402823e+38  3.402823e+38   
    10095  10.998260  27.352158  3.402823e+38  3.402823e+38  3.402823e+38   
    10096  10.999163  27.352068  3.402823e+38  3.402823e+38  3.402823e+38   

           median_canopy  canopy_openness  
0   16347   5.097809e+00     3.

In [10]:
print(len(all_points))

1360892


In [11]:
all_points.drop(all_points[all_points['mean_canopy'] > 100].index, inplace = True)

In [12]:
print(len(all_points))

965508


In [13]:
np.savetxt("D:\\masters_data\\output\\csv\\all_points.csv", all_points, delimiter=",")